# Before We Begin

You will need:
  1. a virtualenv
  2. Jupyter notebook running a kernel in that virtualenv
  3. Klein and Treq (and therefore Twisted) installed in that kernel

To get those:

```console
$ virtualenv abc
$ . abc/bin/activate
$ pip install jupyter[notebook] klein treq
$ jupyter notebook Twisted-Tutorial.ipynb
```

In [1]:
from tornado.platform.twisted import install
reactor = install()

from twisted.web.server import Site
from twisted.internet import endpoints
from klein import route, resource

description = "tcp:8080"

ep = endpoints.serverFromString(reactor, description)
s = Site(resource())
ep.listen(Site(resource()))

<Deferred at 0x10966da28 current result: <<class 'twisted.internet.tcp.Port'> of twisted.web.server.Site on 8080>>

In [2]:
from klein import route
@route('/')
def home(request):
    return 'Hello, everybody!'

In [6]:
@route('/greet/<user>')
def froop(request, user):
    return 'Hello {}! Nice to meet you.'.format(user.capitalize())

In [8]:
import json
from twisted.internet import defer
import treq

@route('/ip')
@defer.inlineCallbacks
def getip(request):
    url = 'https://httpbin.org/ip'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    ip = value['origin']
    defer.returnValue('Your ip is: {}'.format(ip))

In [9]:
import webbrowser
webbrowser.open("http://127.0.0.1:8080/ip")

True

In [10]:
@route('/compare')
@defer.inlineCallbacks
def compare(request):
    url = 'https://httpbin.org/delay/10'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    first_ip = value['origin']
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    second_ip = value['origin']    
    defer.returnValue('These are the same:{}=={}'.format(first_ip, second_ip))

In [11]:
webbrowser.open("http://127.0.0.1:8080/compare")

True

In [12]:
@route('/faster-compare')
def faster_compare(request):
    url = 'https://httpbin.org/delay/10'
    d = treq.get(url)
    d.addCallback(lambda response: response.content())
    d.addCallback(json.loads)
    d.addCallback(lambda dct: dct['origin'])
    d2 = treq.get(url)
    d2.addCallback(lambda response: response.content())
    d2.addCallback(json.loads)
    d2.addCallback(lambda dct: dct['origin'])
    both = defer.gatherResults([d, d2])
    def do_format(ips):
        first_ip, second_ip = ips
        return 'These are the same:{}=={}'.format(first_ip, second_ip)
    both.addCallback(do_format)
    return both

In [13]:
webbrowser.open("http://127.0.0.1:8080/faster-compare")

True

In [28]:
from twisted.internet import task
import operator

@route('/add/<int:a>/<int:b>')
def add(request, a, b):
    d = task.deferLater(reactor, 2, operator.add, a, b)
    # deferLater does not block like time.sleep does
    # reactor, delay(sec), operator, args...
    # Simulate a delay
    d.addCallback(str)
    return d

In [24]:
webbrowser.open("http://127.0.0.1:8080/add/3/4")

True

In [29]:
def remote_add(x, y):
    url = 'http://localhost:8080/add'
    d = treq.get(url + '/{}/{}'.format(x, y))
    d.addCallback(lambda response: response.content())
    return d

@route('/add4/<int:a>/<int:b>/<int:c>/<int:d>')
def add4(request, a, b, c, d):
    d = defer.gatherResults([remote_add(a, b), 
                             remote_add(c, d)])
    @d.addCallback
    def final_add(results):
        x, y = results
        return remote_add(x, y)
    return d

@route('/slow-add4/<int:a>/<int:b>/<int:c>/<int:d>')
@defer.inlineCallbacks
def slow_add4(request, a, b, c, d):
    x = yield remote_add(a, b)
    y = yield remote_add(c, d)
    res = yield remote_add(x, y)
    defer.returnValue(str(res))

In [30]:
webbrowser.open("http://127.0.0.1:8080/add4/3/4/9/14")
webbrowser.open("http://127.0.0.1:8080/slow-add4/3/4/9/14")

True

In [31]:
counter = 0
@route('/count')
def count(request):
    global counter
    counter += 1
    return str(counter)
# Cache parallel, redirect to where the data is already

In [35]:
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")

True

In [40]:
from twisted.internet import task
loops = 0
def add_loop():
    global loops
    loops += 1

@route('/loops')
def show_loops(request):
    return str(loops)

lc = task.LoopingCall(add_loop)
d = lc.start(1)
# Could be a simple celery-like runner

In [41]:
reactor.callLater(6, webbrowser.open, "http://127.0.0.1:8080/loops")
reactor.callLater(12, webbrowser.open, "http://127.0.0.1:8080/loops")

In [65]:
# Chat server
blocking = []

@route('/block')
def block(request):
    result = defer.Deferred()
    blocking.append(result)
    result.addCallback(lambda x: x.upper())
    return result

@route('/unblock/<string:value>')
def unblock(request, value):
    blocking.pop(0).callback(value)
    return u"OK " + value

@route('/block_length')
def blocking_resp(request):
    return 'The length of blocking is {}'.format(len(blocking))


In [68]:
webbrowser.open("http://127.0.0.1:8080/block")

True

In [61]:
webbrowser.open("http://127.0.0.1:8080/unblock/python%20works")

True

In [67]:
webbrowser.open('http://127.0.0.1:8080/block_length')

True